In [ ]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

### TOOL Creation

In [91]:
# Tool Create

@tool
def multiply(a:int,b:int)->int:
    """Given 2 numbers a and b this tool returns their ptoduct"""
    return a * b

In [92]:
print(multiply.invoke({'a':3,'b':5}))
print(multiply.name)
print(multiply.description)
print(multiply.args)

15
multiply
Given 2 numbers a and b this tool returns their ptoduct
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [93]:
# tool calling
llm = ChatGroq(
    model_name='llama3-8b-8192',
    temperature = 0.5,
    streaming=False
)

### TOOL Binding

In [94]:
# Binding tools
llm_with_tools = llm.bind_tools([multiply])
# Not every llm has this capability to bind the tools 

In [95]:
# When and where llm can call the tool

### TOOL Calling

In [96]:
llm_with_tools.invoke("can you multiply 3 with 10")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yfyn', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 910, 'total_tokens': 982, 'completion_time': 0.06, 'prompt_time': 0.169961119, 'queue_time': 0.256547712, 'total_time': 0.229961119}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f243f4fb-f488-431e-b479-1afb67b39cf8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_yfyn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 910, 'output_tokens': 72, 'total_tokens': 982})

In [97]:
llm_with_tools.invoke("can you multiply 3 with 10").tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'call_pfsk',
  'type': 'tool_call'}]

In [98]:
llm_with_tools.invoke("can you multiply 3 with 10").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_70pd',
 'type': 'tool_call'}

In [99]:
result = llm_with_tools.invoke("can you multiply 3 with 10")

In [100]:
result.tool_calls[0]['args']

{'a': 3, 'b': 10}

In [101]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_qyj4',
 'type': 'tool_call'}

### TOOL Execution

In [102]:
multiply.invoke(result.tool_calls[0]['args'])

30

### Tool Message
- We can sent this tool message to llm and LLM will generate the message

In [103]:
multiply.invoke(
    {'name': 'multiply',
    'args': {'a': 3, 'b': 10},
    'id': 'call_z4mm',
    'type': 'tool_call'}
)

ToolMessage(content='30', name='multiply', tool_call_id='call_z4mm')

In [104]:
from langchain_core.messages import HumanMessage

In [105]:
query = HumanMessage("can you multiply 7 with 5")

In [106]:
messages = [query]

In [107]:
messages

[HumanMessage(content='can you multiply 7 with 5', additional_kwargs={}, response_metadata={})]

In [108]:
result = llm_with_tools.invoke(messages)

In [109]:
messages.append(result)

In [110]:
messages

[HumanMessage(content='can you multiply 7 with 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9g4k', 'function': {'arguments': '{"a":7,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 910, 'total_tokens': 982, 'completion_time': 0.06, 'prompt_time': 0.170856164, 'queue_time': 0.26651569399999997, 'total_time': 0.230856164}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--69b41588-ab59-4f33-962f-a44cf775b71f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 7, 'b': 5}, 'id': 'call_9g4k', 'type': 'tool_call'}], usage_metadata={'input_tokens': 910, 'output_tokens': 72, 'total_tokens': 982})]

In [111]:
tool_result = multiply.invoke(result.tool_calls[0])

In [112]:
messages.append(tool_result)

In [113]:
# Maintaining the conversational History
messages

[HumanMessage(content='can you multiply 7 with 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9g4k', 'function': {'arguments': '{"a":7,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 910, 'total_tokens': 982, 'completion_time': 0.06, 'prompt_time': 0.170856164, 'queue_time': 0.26651569399999997, 'total_time': 0.230856164}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--69b41588-ab59-4f33-962f-a44cf775b71f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 7, 'b': 5}, 'id': 'call_9g4k', 'type': 'tool_call'}], usage_metadata={'input_tokens': 910, 'output_tokens': 72, 'total_tokens': 982}),
 ToolMessage(content='35', name='multiply', tool_call_id='call_9g4k')]

In [ ]:
# After maintaining conversational history we feed it to the llm
llm_with_tools.invoke(messages)


AIMessage(content='35', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 981, 'total_tokens': 983, 'completion_time': 0.001666667, 'prompt_time': 0.121812234, 'queue_time': 0.257826097, 'total_time': 0.123478901}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run--15aecca2-2286-45c4-9604-c2c9a121dbd9-0', usage_metadata={'input_tokens': 981, 'output_tokens': 2, 'total_tokens': 983})

In [115]:
llm_with_tools.invoke(messages).content

'35'